In [2]:
!pip install pika

In [3]:
import pika
import json

# Параметры подключения
RABBITMQ_HOST = '192.168.0.18'  # или IP-адрес сервера
RABBITMQ_PORT = 8006  # ваш нестандартный порт
EXCHANGE_NAME = 'amq.topic'  # или имя вашего exchange
ROUTING_KEY = 'incoming_json'  # ключ маршрутизации
QUEUE_NAME = 'your_queue_name'  # имя очереди (если нужно)

# Тестовые данные
test_message = {
    "id": 123,
    "name": "Тестовое сообщение",
    "timestamp": "2024-01-15T10:30:00Z",
    "data": {
        "value": 42.5,
        "status": "success"
    }
}

def send_json_to_rabbitmq():
    try:
        # 1. Подключение к RabbitMQ
        credentials = pika.PlainCredentials('guest', 'guest')  # замените на свои
        parameters = pika.ConnectionParameters(
            host=RABBITMQ_HOST,
            port=RABBITMQ_PORT,
            credentials=credentials,
            heartbeat=600,
            blocked_connection_timeout=300
        )
        
        connection = pika.BlockingConnection(parameters)
        channel = connection.channel()
        
        # 2. Объявление exchange (если нужно)
        # channel.exchange_declare(
        #     exchange=EXCHANGE_NAME,
        #     exchange_type='topic',
        #     durable=True
        # )
        
        # 3. Отправка сообщения
        channel.basic_publish(
            exchange=EXCHANGE_NAME,  # используйте '' для default exchange
            routing_key=ROUTING_KEY,
            body=json.dumps(test_message, ensure_ascii=False),
            properties=pika.BasicProperties(
                content_type='application/json',
                delivery_mode=2,  # persistent message
                headers={'type': 'test_message'}
            )
        )
        
        print(f"✅ Сообщение отправлено в RabbitMQ ({RABBITMQ_HOST}:{RABBITMQ_PORT})")
        print(f"   Exchange: {EXCHANGE_NAME}")
        print(f"   Routing key: {ROUTING_KEY}")
        print(f"   Тело сообщения: {json.dumps(test_message, indent=2, ensure_ascii=False)}")
        
        # 4. Закрытие соединения
        connection.close()
        
    except Exception as e:
        print(f"❌ Ошибка при отправке сообщения: {str(e)}")



In [7]:
def check_rabbitmq_status():
    """Проверяет состояние RabbitMQ и наличие очередей"""
    try:
        credentials = pika.PlainCredentials('guest', 'guest')
        parameters = pika.ConnectionParameters(
            host=RABBITMQ_HOST,
            port=RABBITMQ_PORT,
            credentials=credentials,
            connection_attempts=3,
            retry_delay=2
        )
        
        connection = pika.BlockingConnection(parameters)
        channel = connection.channel()
        
        print("=== Статус RabbitMQ ===")
        print(f"Подключено к {RABBITMQ_HOST}:{RABBITMQ_PORT}")
        
        # Пытаемся получить информацию о сервере
        try:
            # Проверяем connection
            if connection.is_open:
                print("✅ Соединение установлено")
                
                # Создаем тестовую очередь, если не существует
                channel.queue_declare(queue='test_queue', durable=True, exclusive=False)
                
                # Привязываем к exchange amq.topic
                channel.queue_bind(
                    exchange='amq.topic',
                    queue='test_queue',
                    routing_key='incoming_json'
                )
                
                print("✅ Очередь 'test_queue' создана и привязана к amq.topic")
                print("✅ Routing key: 'incoming_json'")
                
                # Отправляем тестовое сообщение
                channel.basic_publish(
                    exchange='amq.topic',
                    routing_key='incoming_json',
                    body='{"test": "message"}',
                    properties=pika.BasicProperties(
                        delivery_mode=2,
                        content_type='application/json'
                    )
                )
                
                print("✅ Тестовое сообщение отправлено")
                
                # Получаем сообщение (чтобы очистить очередь)
                method_frame, header_frame, body = channel.basic_get(queue='test_queue')
                if method_frame:
                    print(f"✅ Получено сообщение: {body.decode()}")
                    channel.basic_ack(method_frame.delivery_tag)
                else:
                    print("❌ Сообщение не получено (возможно, не дошло до очереди)")
                    
        except Exception as e:
            print(f"❌ Ошибка при проверке: {e}")
            
        connection.close()
        
    except pika.exceptions.AMQPConnectionError as e:
        print(f"❌ Не удалось подключиться к RabbitMQ: {e}")
        print("Проверьте:")
        print(f"1. Доступен ли сервер {RABBITMQ_HOST}:{RABBITMQ_PORT}")
        print("2. Правильные ли учетные данные")
        print("3. Разрешен ли доступ с вашего IP")
    except Exception as e:
        print(f"❌ Общая ошибка: {e}")


check_rabbitmq_status()

=== Статус RabbitMQ ===
Подключено к 192.168.0.18:8006
✅ Соединение установлено
✅ Очередь 'test_queue' создана и привязана к amq.topic
✅ Routing key: 'incoming_json'
✅ Тестовое сообщение отправлено
✅ Получено сообщение: {"test": "message"}


In [6]:
send_json_to_rabbitmq()

✅ Сообщение отправлено в RabbitMQ (192.168.0.18:8006)
   Exchange: amq.topic
   Routing key: incoming_json
   Тело сообщения: {
  "id": 123,
  "name": "Тестовое сообщение",
  "timestamp": "2024-01-15T10:30:00Z",
  "data": {
    "value": 42.5,
    "status": "success"
  }
}
